# Verificacion de formato para cada .csv para que puedan ser transferibles a formato DarwinCore

In [1]:
import os
import re
import pandas as pd

# Los archivos csv se deben encontrar en la carpeta llamada 'Detecciones'
ruta_carpeta = './Detecciones_crudas/'

# Crear lista de archivos .csv en la carpeta
csv_files = [os.path.join(ruta_carpeta, f) for f in os.listdir(ruta_carpeta) if f.endswith('.csv')]


### Verificacion de columnas


En la siguiente funcion se buscan organizar y estandarizar todas las bases de datos incluidas en el proyecto, para ello se:<br>
<br>
1. Cambia el nombre a las siguientes columnas, si esta no existe, se crea y se llena con "NA":<br>
Start_time (s) a startTime<br>
End_time (s) a endTime<br>
Clements2024_Scientific_Name a speciesName<br>
Confidence a confidence<br>
original_file a filePath<br>
punto a recorderID<br>
project a projectName<br>
aru_type a recorderModel<br>
recorder a setupBy<br>
segmentfile a segmentID<br>
<br>
2. Se hacen las siguientes transformaciones:<br>
Se deja el nombre del archivo (sin la ruta completa) a mediaID<br>
Si la columna deploymentGroups no existe, crea una nueva llamada deploymentGroups llenandola con "NA"<br>
Verifica que latitud y longitus estan en formato de coordenadas geograficas<br>
Verifica que los valores de confianza estan entre 0 y 1<br>
Crea la timestamp y convierte en formato YYYY-MM-DD HH:MM:SS<br>


In [2]:
def standardize_dataframe(df):
    """
    Standardizes a given detection DataFrame according to the specs provided.
    Steps:
        1. Rename columns, add missing if needed.
        2. Apply required transformations.
        3. Ensure and order required columns.
        4. Rename segment_file as segmentID.
    """

    # Rename "original_file" or "file" to "filePath" (do this before anything else)
    if 'original_file' in df.columns:
        df.rename(columns={'original_file': 'filePath'}, inplace=True)
    elif 'File' in df.columns:
        df.rename(columns={'File': 'filePath'}, inplace=True)
    if 'filePath' not in df.columns:
        df['filePath'] = "NA"

    # Map of current to new column names, REMOVE 'original_file': 'mediaID' mapping.
    rename_map = {
        'Start_time (s)': 'startTime',
        'End_time (s)': 'endTime',
        'Clements2024_Scientific_Name': 'speciesName',
        'Confidence': 'confidence',
        'punto': 'recorderID',
        'project': 'projectName',
        'aru_type': 'recorderModel',
        'recorder': 'setupBy'
    }

    # Apply renaming for the rest
    for old, new in rename_map.items():
        if old in df.columns:
            df.rename(columns={old: new}, inplace=True)
        if new not in df.columns:
            df[new] = "NA"

    # Now, create mediaID as the basename of filePath, place as second column
    df['mediaID'] = df['filePath'].astype(str).apply(lambda x: os.path.basename(x) if x != "NA" else "NA")

    # Add classifiedBy, deploymentStart, deploymentEnd, deploymentGroups, locality, municipality if not present
    for col in ['classifiedBy', 'deploymentStart', 'deploymentEnd', 'deploymentGroups', 'locality', 'municipality']:
        if col not in df.columns:
            df[col] = "NA"
                
    # --- Step 2: Transformations ---
    # deploymentGroups: If not existing, already set to 'NA' above

    # latitude & longitude should be float and valid
    for lat_long_col in ['latitude', 'longitude']:
        if lat_long_col in df.columns:
            df[lat_long_col] = pd.to_numeric(df[lat_long_col], errors='coerce')
            # Optionally: Check/clip to valid ranges
            if lat_long_col == 'latitude':
                df.loc[(df[lat_long_col] < -90) | (df[lat_long_col] > 90), lat_long_col] = pd.NA
            else:
                df.loc[(df[lat_long_col] < -180) | (df[lat_long_col] > 180), lat_long_col] = pd.NA

    # Make sure confidence is a number between 0 and 1
    if 'confidence' in df.columns:
        df['confidence'] = pd.to_numeric(df['confidence'], errors='coerce')
        # If it's, e.g., percentage, convert
        if df['confidence'].max() > 1:
            # Assume it's 0-100, convert
            df['confidence'] = df['confidence'] / 100.0
        # Force in range
        df.loc[df['confidence'] < 0, 'confidence'] = 0
        df.loc[df['confidence'] > 1, 'confidence'] = 1

    # --- timestamp: from mediaID ---
    def extract_timestamp(media_id):
        # Expects format like G6370_20240212_103000.WAV or similar
        if not isinstance(media_id, str):
            return "NA"
        # Remove extension
        fname = os.path.splitext(media_id)[0]
        # Split by "_"
        parts = fname.split("_")
        if len(parts) < 2:
            return "NA"
        # Last two parts supposed to be e.g. 20240212, 103000
        date_str = parts[-2]
        time_str = parts[-1]
        date_str = re.sub(r'[^\d]', '', date_str)   # clean non-digit
        time_str = re.sub(r'[^\d]', '', time_str)
        timestamp = f"{date_str}_{time_str}"
        return timestamp

    df['timestamp'] = df['mediaID'].apply(extract_timestamp)

    # Convert timestamp to 'YYYY-MM-DD HH:MM:SS'
    def reformat_timestamp(ts):
        # Expects 'YYYYMMDD_HHMMSS' e.g. 20240212_103000
        if isinstance(ts, str) and len(ts) >= 15 and "_" in ts:
            date_part, time_part = ts.split("_")
            if len(date_part) == 8 and len(time_part) == 6:
                return f"{date_part[:4]}-{date_part[4:6]}-{date_part[6:8]} {time_part[:2]}:{time_part[2:4]}:{time_part[4:6]}"
        return pd.NA

    df['timestamp'] = df['timestamp'].apply(reformat_timestamp)

    # --- Step 3: Ensure columns exist and sort ---

    # Update required_columns: add "filePath" as FIRST, "mediaID" as SECOND
    required_columns = [
        'filePath', 'mediaID','startTime', 'endTime', 'speciesName', 'confidence',
        'classifiedBy', 'recorderID', 'recorderModel', 'locality',
        'projectName', 'municipality','latitude', 'longitude', 'setupBy', 'deploymentStart',
        'deploymentEnd', 'deploymentGroups','timestamp'
    ]

    for col in required_columns:
        if col not in df.columns:
            df[col] = "NA"

    # Handle 'segment_file' as 'segmentID', drop habitat if it exists
    # If 'segmentID' already exists, do nothing.
    if 'segmentID' not in df.columns:
        if 'segment_file' in df.columns:
            df.rename(columns={'segment_file': 'segmentID'}, inplace=True)
        else:
            df['segmentID'] = "NA"
    if 'habitat' in df.columns:
        df = df.drop(columns=['habitat'])

    # Ensure all required columns in given order, fill missing (already done above)
    # Now append 'segmentID' at the end
    output_columns = required_columns + ['segmentID']
    for col in output_columns:
        if col not in df.columns:
            df[col] = "NA"
    df = df[output_columns]

    return df


In [3]:
for file in csv_files:
    df = pd.read_csv(file)
    df = standardize_dataframe(df)
    output_folder = "./Detecciones_estandarizadas"
    output_path = os.path.join(output_folder, os.path.basename(file))
    df.to_csv(output_path, index=False)